In [1]:
import function as f
import pandas as pd

In [2]:
# Load the model
my_model = f.load_model()

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from resources/llama-2-7b-chat.gguf.q5_K_S.bin (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:               output_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:                    output.weight q6_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_q.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.attn_k.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:              blk.0.attn_v.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    6:         blk.0.attn_output.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:           blk.0.attn_norm.weight f32      [  4096,     1, 

In [3]:
# Create the template
 
template = """\
    name: {name}
    elements: {elements}
    classification : {classification}\
    """
my_template = f.create_prompt(template, ["name","elements","classification"])

In [22]:
# Create the few shot Prompt 

 # 1- The prompt
prompt = "\n\n Please determine whether the provided attribute is considered sensitive or not, considering the regulatory context of Morocco under its Data Protection laws. Provide only the classification without explanations.\n\nSensitive -> An attribute is considered sensitive if the data subject has a reasonable expectation of privacy regarding this information, or if the attribute is inherently sensitive in nature (e.g., financial information).\n\nNot Sensitive -> An attribute is considered not sensitive if the data subject does not have a reasonable expectation of privacy regarding this information, and if the attribute is not inherently sensitive by nature (e.g., demographic information).\n\n Please ensure that your classification is based on both the attribute name and the considerations of the instances provided, as sometimes the name of the attribute may not convey its sensitivity.\n Please refrain from reflexively classifying attributes as not sensitive. Instead, carefully weigh the context, privacy expectations of each attribute.\n\n Examples:\n"


 # 2- The examples
 
classification_examples = [ {"name": "Name","elements" : "Amine,issam,Hamza", "classification" : "Sensitive"},
                            {"name": "Age", "elements" : "20,29,24", "classification" : "NOT Sensitive"},
                            {"name": "Income", "elements" : " 25K,13K,18K", "classification" : "Sensitive"},
                            {"name": "Credit Card","elements" : "321031399,203193842,301384174", "classification" : "Sensitive"},
                            {"name": "City","elements" : "london,madrid,roma", "classification" : "NOT Sensitive"},
                            {"name": "Phone Number","elements" : "20132842,30139201,43042120", "classification" : "Sensitive"},
    
                            ]
    # 3- The suffix 
suffix = " name: {name} \n elements: {elements} \n classification :"

    # 4- The input variables
input_variables=["name","elements"]
    
    # 5- The example separator
example_separator="\n\n"


my_few_shot_prompt = f.create_few_shot_prompt(classification_examples=classification_examples,prompt= my_template,input_var=input_variables,
                                              prefix=prompt,suffix=suffix,sep=example_separator)


In [23]:
test_data = pd.read_csv("resources/data/adult.csv")

In [24]:
input_dict = f.get_func_variables_from_pandas(test_data)
keys = input_dict.keys()

In [25]:
my_df = pd.DataFrame(columns=["Attribute","Classification"])

for elem in keys:
    print(my_few_shot_prompt.format(name=elem, elements=','.join(map(str, input_dict[elem]))))

    result = f.classify_attribute(attribute_name = elem, attribute_instances= ','.join(map(str, input_dict[elem])), few_shot_prompt = my_few_shot_prompt, model=my_model)
    df2 = pd.DataFrame([[elem, result]], columns=['Attribute', 'Classification'])
    
    my_df = pd.concat([my_df, df2], ignore_index=True)
    



 Please determine whether the provided attribute is considered sensitive or not, considering the regulatory context of Morocco under its Data Protection laws. Provide only the classification without explanations.

Sensitive -> An attribute is considered sensitive if the data subject has a reasonable expectation of privacy regarding this information, or if the attribute is inherently sensitive in nature (e.g., financial information).

Not Sensitive -> An attribute is considered not sensitive if the data subject does not have a reasonable expectation of privacy regarding this information, and if the attribute is not inherently sensitive by nature (e.g., demographic information).

 Please ensure that your classification is based on both the attribute name and the considerations of the instances provided, as sometimes the name of the attribute may not convey its sensitivity.
 Please refrain from reflexively classifying attributes as not sensitive. Instead, carefully weigh the context, pr

Llama.generate: prefix-match hit


 NOT Sensitive
Please classify the following attribute based on the above considerations.
age
Please let me know if you need any further clarification or details.

 Please determine whether the provided attribute is considered sensitive or not, considering the regulatory context of Morocco under its Data Protection laws. Provide only the classification without explanations.

Sensitive -> An attribute is considered sensitive if the data subject has a reasonable expectation of privacy regarding this information, or if the attribute is inherently sensitive in nature (e.g., financial information).

Not Sensitive -> An attribute is considered not sensitive if the data subject does not have a reasonable expectation of privacy regarding this information, and if the attribute is not inherently sensitive by nature (e.g., demographic information).

 Please ensure that your classification is based on both the attribute name and the considerations of the instances provided, as sometimes the name o


llama_print_timings:        load time =  6059.88 ms
llama_print_timings:      sample time =    29.09 ms /    40 runs   (    0.73 ms per token,  1375.00 tokens per second)
llama_print_timings: prompt eval time =  3645.50 ms /    41 tokens (   88.91 ms per token,    11.25 tokens per second)
llama_print_timings:        eval time =  1859.62 ms /    39 runs   (   47.68 ms per token,    20.97 tokens per second)
llama_print_timings:       total time =  5610.78 ms
Llama.generate: prefix-match hit


 NOT Sensitive 
Please provide the attribute name and the corresponding classification (Sensitive or Not Sensitive) for each instance provided.

 Please determine whether the provided attribute is considered sensitive or not, considering the regulatory context of Morocco under its Data Protection laws. Provide only the classification without explanations.

Sensitive -> An attribute is considered sensitive if the data subject has a reasonable expectation of privacy regarding this information, or if the attribute is inherently sensitive in nature (e.g., financial information).

Not Sensitive -> An attribute is considered not sensitive if the data subject does not have a reasonable expectation of privacy regarding this information, and if the attribute is not inherently sensitive by nature (e.g., demographic information).

 Please ensure that your classification is based on both the attribute name and the considerations of the instances provided, as sometimes the name of the attribute may

...


llama_print_timings:        load time =  6059.88 ms
llama_print_timings:      sample time =    13.87 ms /    19 runs   (    0.73 ms per token,  1369.67 tokens per second)
llama_print_timings: prompt eval time =   324.89 ms /    43 tokens (    7.56 ms per token,   132.35 tokens per second)
llama_print_timings:        eval time =   851.39 ms /    18 runs   (   47.30 ms per token,    21.14 tokens per second)
llama_print_timings:       total time =  1222.29 ms
Llama.generate: prefix-match hit


 NOT Sensitive    
 Please answer for the following attributes:
 Amine,Iss

 Please determine whether the provided attribute is considered sensitive or not, considering the regulatory context of Morocco under its Data Protection laws. Provide only the classification without explanations.

Sensitive -> An attribute is considered sensitive if the data subject has a reasonable expectation of privacy regarding this information, or if the attribute is inherently sensitive in nature (e.g., financial information).

Not Sensitive -> An attribute is considered not sensitive if the data subject does not have a reasonable expectation of privacy regarding this information, and if the attribute is not inherently sensitive by nature (e.g., demographic information).

 Please ensure that your classification is based on both the attribute name and the considerations of the instances provided, as sometimes the name of the attribute may not convey its sensitivity.
 Please refrain from reflexively classif


llama_print_timings:        load time =  6059.88 ms
llama_print_timings:      sample time =    15.07 ms /    20 runs   (    0.75 ms per token,  1327.32 tokens per second)
llama_print_timings: prompt eval time =   473.58 ms /    69 tokens (    6.86 ms per token,   145.70 tokens per second)
llama_print_timings:        eval time =   900.68 ms /    19 runs   (   47.40 ms per token,    21.10 tokens per second)
llama_print_timings:       total time =  1428.22 ms
Llama.generate: prefix-match hit


 Sensitive 
Please provide the attribute name and its corresponding classification (Sensitive or Not Sensitive) based on the provided instances.


llama_print_timings:        load time =  6059.88 ms
llama_print_timings:      sample time =    22.87 ms /    31 runs   (    0.74 ms per token,  1355.25 tokens per second)
llama_print_timings: prompt eval time =   339.96 ms /    58 tokens (    5.86 ms per token,   170.61 tokens per second)
llama_print_timings:        eval time =  1430.34 ms /    30 runs   (   47.68 ms per token,    20.97 tokens per second)
llama_print_timings:       total time =  1846.35 ms


In [26]:
my_df

,Attribute,Classification
0,age,NOT Sensitive
1,workclass,NOT Sensitive
2,fnlwgt,NOT Sensitive
3,education,NOT Sensitive
4,educational-num,NOT Sensitive
5,marital-status,Sensitive
6,occupation,NOT Sensitive
7,relationship,NOT Sensitive
8,race,NOT Sensitive
9,gender,NOT Sensitive
